André Luiz da Silva Conde

<center>

# Filtragem no domínio da frequência

</center>

1. Calcule e visualize o espectro de uma imagem 512x512 pixels:

1.a crie e visualize uma imagem simples – quadrado branco sobre fundo preto

In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
from PIL import Image


def make_square(shape: (int, int), inner_shape: (int, int, int), translate: (int, int) = (0, 0)) -> np.array:
    if inner_shape[0] > shape[0] or inner_shape[1] > shape[1]:
        raise ValueError('shape_center must be inside shape')

    square = np.zeros(shape)
    x_center = shape[0] / 2 + translate[0]
    y_center = shape[1] / 2 + translate[1]

    square[int(x_center - inner_shape[0] / 2):int(x_center + inner_shape[0] / 2),
    int(y_center - inner_shape[1] / 2):int(y_center + inner_shape[1] / 2)] = 255

    pillow = Image.fromarray(square)
    rotated = pillow.rotate(inner_shape[2])
    square = np.array(rotated)

    return square


square_shape = (512, 512)
square_center_shape = (64, 64, 180)


def get_default_square():
    return make_square(square_shape, square_center_shape)


plt.imshow(make_square(square_shape, square_center_shape), cmap='gray')
plt.show()

1.b calcular e visualizar seu espectro de Fourier (amplitudes).

In [ ]:
def fourier_transform_unshift(img):
    # Perform Fourier Transform
    dft = np.fft.fft2(img)

    # Compute magnitude and phase spectrum
    fourier = 20 * np.log(np.abs(dft) + 1e-8)
    phase = np.angle(dft)

    return fourier, phase


square_fourier_unshift, square_phase_unshift = fourier_transform_unshift(get_default_square())

plt.imshow(square_fourier_unshift, cmap='gray')
plt.show()

1.c calcular e visualizar seu espectro de Fourier (fases).

In [ ]:
plt.imshow(square_phase_unshift, cmap='gray')
plt.show()

1.d obter e visualizar seu espectro de Fourier centralizado

In [ ]:
def fourier_transform(img):
    # Perform Fourier Transform
    dft = np.fft.fft2(img)
    dft_shift = np.fft.fftshift(dft)

    # Compute magnitude and phase spectrum
    fourier = 20 * np.log(np.abs(dft_shift) + 1e-8)
    phase = np.angle(dft_shift)

    return fourier, phase


square_fourier, _ = fourier_transform(get_default_square())

In [ ]:

plt.imshow(square_fourier, cmap='gray')
plt.show()

1.e Aplique uma rotação de 40º no quadrado e repita os passo b-d;

In [ ]:
square_center_shape = (64, 64, 40)

_, axs = plt.subplots(1, 3, figsize=(15, 15))

axs[0].imshow(make_square(square_shape, square_center_shape), cmap='gray')
axs[0].set_title('Imagem original')

square_fourier_rotated, square_phase_rotated = fourier_transform(make_square(square_shape, square_center_shape))

axs[1].imshow(square_fourier_rotated, cmap='gray')
axs[1].set_title('Espectro de Fourier')

axs[2].imshow(square_phase_rotated, cmap='gray')
axs[2].set_title('Fase')

plt.show()


1.f Aplique uma translação nos eixos x e y no quadrado e repita os passo b-d

In [ ]:
translate = (100, 100)
inner_shape = (64, 64, 0)

square_translated = make_square(square_shape, inner_shape, translate)

_, axs = plt.subplots(1, 3, figsize=(15, 15))

axs[0].imshow(square_translated, cmap='gray')
axs[0].set_title('Imagem original')

square_fourier_translated, square_phase_translated = fourier_transform(square_translated)

axs[1].imshow(square_fourier_translated, cmap='gray')
axs[1].set_title('Espectro de Fourier')

axs[2].imshow(square_phase_translated, cmap='gray')
axs[2].set_title('Fase')

plt.show()


1.g Aplique um zoom na imagem e repita os passo b-d.

In [ ]:
# the zoom is made by changing the inner shape of the square
# zooming in is done by making the inner shape smaller
zoom = 3
square_center_shape = (64 * zoom, 64 * zoom, 0)

_, axs = plt.subplots(1, 3, figsize=(15, 15))

axs[0].imshow(make_square(square_shape, square_center_shape), cmap='gray')
axs[0].set_title('Imagem original')

square_fourier_zoomed, square_phase_zoomed = fourier_transform(make_square(square_shape, square_center_shape))

axs[1].imshow(square_fourier_zoomed, cmap='gray')
axs[1].set_title('Espectro de Fourier')

axs[2].imshow(square_phase_zoomed, cmap='gray')
axs[2].set_title('Fase')

plt.show()

1.h Explique o que acontece com a transformada de Fourier quando é aplicado a rotação, translação e zoom.

- Rotação: A rotação da imagem no domínio espacial causa uma rotação no domínio da frequência. Isso acontece porque a rotação no domínio espacial é feita por meio de uma multiplicação por uma matriz de rotação, que é equivalente a uma convolução no domínio da frequência. A convolução no domínio da frequência é equivalente à multiplicação no domínio espacial, então a rotação no domínio espacial é equivalente a uma multiplicação no domínio da frequência.

- Translação: A translação da imagem no domínio espacial causa uma multiplicação por uma exponencial no domínio da frequência. Isso acontece porque a translação no domínio espacial é feita por meio de uma multiplicação por uma exponencial, equivalente a uma convolução no domínio da frequência. A convolução no domínio da frequência é equivalente à multiplicação no domínio espacial, então a translação no domínio espacial é equivalente a uma multiplicação no domínio da frequência.

- Zoom: O zoom da imagem no domínio espacial causa uma convolução no domínio da frequência. Isso acontece porque o zoom no domínio espacial é feito por meio de uma convolução, equivalente a uma multiplicação no domínio da frequência. A convolução no domínio da frequência é equivalente à multiplicação no domínio espacial, então o zoom no domínio espacial é equivalente a uma multiplicação no domínio da frequência.

2. Crie filtros passa-baixa do tipo ideal, butterworth e gaussiano e aplique-o às imagens disponibilizadas. Visualize o seguinte:

A transformada inversa de Fourier do filtro é dada por:

In [ ]:
def inverse_fourier_transform(img, phase):
    # Compute inverse Fourier Transform
    magnitude_spectrum = np.exp((img - 1e-8) / 20)
    real = magnitude_spectrum * np.cos(phase)
    imag = magnitude_spectrum * np.sin(phase)
    dft_shift = real + imag * 1j
    dft = np.fft.ifftshift(dft_shift)
    img_back = np.fft.ifft2(dft)

    return img_back

Para os cálculos dos filtros foi utilizada uma função auxiliar para calcular a distância entre dois pontos. A função recebe a forma da imagem e as coordenadas dos pontos e retorna a distância entre eles no domínio da frequência. A implementação da função é mostrada abaixo:

In [ ]:
def dij(shape, i, j):
    return np.sqrt((i - shape[0] / 2) ** 2 + (j - shape[1] / 2) ** 2)

Abaixo estão as implementações dos filtros passa-baixa ideal, butterworth e gaussiano. As implementações são baseadas nas equações apresentadas em aula.
Dado D(u, v) como:
<br>
<center>

$D(u, v) = sqrt((u - M/2)^2 + (v - N/2)^2)$

</center>

e  H( u, v) para o filtro passa baixa ideal como:

<center>

$H(u, v) = 1, se D(u, v) <= D_0$ <br>
$H(u, v) = 0, se D(u, v) > D_0$

</center>

segue a implementação do filtro passa-baixa ideal:

In [ ]:
def low_pass_ideal_filter(img, cutoff):
    H = np.zeros(img.shape)

    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            Dij = dij(img.shape, i, j)
            if Dij <= cutoff:
                H[i, j] = 1

    return H

Para o filtro butterworth, a equação é dada por:
<br>
<center>

$H(u, v) = \frac{1}{1 + (\frac{D(u, v)}{D_0})^{2n}}$

</center>

onde n é a ordem do filtro. A implementação do filtro butterworth é mostrada abaixo:

In [ ]:
def low_pass_butterworth_filter(img, cutoff, order):
    H = np.zeros(img.shape)

    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            Dij = dij(img.shape, i, j)
            H[i, j] = 1 / (1 + (Dij / cutoff) ** (2 * order))

    return H

Para o filtro gaussiano, a equação é dada por:
<br>
<center>

$H(u, v) = e^{-\frac{D(u, v)^2}{2D_0^2}}$

</center>

A implementação do filtro gaussiano é mostrada abaixo:

In [ ]:
def low_pass_gaussian_filter(img, cutoff):
    H = np.zeros(img.shape)

    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            Dij = dij(img.shape, i, j)
            H[i, j] = np.exp(-(Dij ** 2) / (2 * (cutoff ** 2)))

    return H

Para facilitar a aplicação dos filtros, foi criada uma função que recebe o tipo do filtro e retorna uma função que aplica o filtro. Essa implementação baseia-se no padrão de projeto Factory. A implementação da função é mostrada abaixo:

In [ ]:
# Low pass filter factory
def make_low_pass_filter(filter_type):
    __name__ = 'filtro passa-baixa'

    if filter_type == 'butterworth':
        return low_pass_butterworth_filter
    elif filter_type == 'gaussian':
        return low_pass_gaussian_filter
    elif filter_type == 'ideal':
        return low_pass_ideal_filter
    else:
        raise Exception('Invalid filter type')

Por fim, a função `apply_filter` aplica o filtro na imagem baseada na banda de passagem. A implementação da função é mostrada abaixo:

In [ ]:
pass_band_types = ['low', 'high']
filter_types = ['butterworth', 'gaussian', 'ideal']


def apply_filter(img, cutoff: int, filter_factory, filter_type: str, order=None):
    if filter_type not in filter_types:
        raise Exception('Invalid filter type')

    f = filter_factory(filter_type)

    if filter_type == 'butterworth':
        if order is None:
            order = static_order

        return f(img, cutoff, order)

    return f(img, cutoff)

A função `process_image` é responsável por aplicar o filtro na imagem e mostrar os resultados. Essa e às demais funções implementadas são utilizadas para processar as imagens com os bandas passa baixa e passa alta. A implementação da função é mostrada abaixo:

In [ ]:
def process_image(img_path: str, cutoff: int, filter_factory, filter_type: str, order=None):
    img = np.array(Image.open('./images/' + img_path).convert('L'))

    F, phase = fourier_transform(img)

    H = apply_filter(img, cutoff, filter_factory, filter_type, order)

    G = F * H

    g = inverse_fourier_transform(G, phase).astype(np.uint8)

    fig, axis = plt.subplots(2, 3)

    plt.suptitle(f'Imagem {img_path}; {filter_type} com frequência de corte {cutoff}')

    axis[0][0].imshow(img, cmap='gray')
    axis[0][0].set_title('f(x, y)')

    axis[0][1].imshow(F, cmap='gray')
    axis[0][1].set_title('F(u, v)')

    axis[0][2].imshow(H, cmap='gray')
    axis[0][2].set_title('H(u, v)')

    axis[1][0].imshow(G, cmap='gray')
    axis[1][0].set_title('F(u, v) * H(u, v)')

    axis[1][1].imshow(g, cmap='gray')
    axis[1][1].set_title('g(x, y)')

    axis[1][2].imshow(img - g, cmap='gray')
    axis[1][2].set_title('f(x, y) - g(x, y)')

    plt.show()


static_order = 1

Aplica o filtro passa-baixa nas imagens com frequência de corte 25 e ordem 1.

In [ ]:
for img_path in os.listdir('./images'):
    for filter_type in filter_types:
        process_image(img_path, 25, make_low_pass_filter, filter_type)

3. Crie um filtro passa-alta do tipo ideal, butterworth e gaussiano e aplique-o às imagens disponibilizadas. Visualize os mesmos dados da tarefa anterior

Para o filtro passa-alta ideal, a equação é dada por:
<br>
<center>

$H(u, v) = 1 - H_{pb}(u, v)$

</center>

 Observando $H(u, v)$ é possível concluir que o filtro passa-alta ideal é o filtro passa-baixa ideal subtraído de 1. A implementação do filtro passa-alta ideal é mostrada abaixo:

In [ ]:
def high_pass_ideal_filter(img, cutoff):
    return 1 - low_pass_ideal_filter(img, cutoff)

Para o filtro passa-alta butterworth, a equação é dada por:
<br>
<center>

$H(u, v) = \frac{1}{1 + (\frac{D_0}{D(u, v)})^{2n}}$

</center>

onde n é a ordem do filtro e $D_0$ é a frequência de corte do filtro. A implementação do filtro passa-alta butterworth é mostrada abaixo:

In [ ]:
def high_pass_butterworth_filter(img, cutoff, order):
    H = np.zeros(img.shape)

    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            Dij = dij(img.shape, i, j)
            H[i, j] = 1 / (1 + (cutoff / Dij) ** (2 * order))

    return H

Para o filtro passa-alta gaussiano, a equação é dada por:
<br>
<center>

$H(u, v) = 1 - e^{-\frac{D(u, v)^2}{2D_0^2}}$

</center>

onde $D_0$ é a frequência de corte do filtro. Mediante análise, conclui-se que o filtro passa-alto gaussiano é o filtro passa-baixa gaussiano subtraído de 1. A implementação do filtro passa-alto gaussiano é mostrada abaixo:


In [ ]:
def high_pass_gaussian_filter(img, cutoff):
    return 1 - low_pass_gaussian_filter(img, cutoff)

The high pass filter factory is shown below:

In [ ]:
def make_high_pass_filter(filter_type):
    __name__ = filter_type

    if filter_type == 'butterworth':
        return high_pass_butterworth_filter
    elif filter_type == 'gaussian':
        return high_pass_gaussian_filter
    elif filter_type == 'ideal':
        return high_pass_ideal_filter
    else:
        raise Exception('Invalid filter type')

In [ ]:
for filter_type in filter_types:
    for img_path in os.listdir('./images'):
        process_image(img_path, 25, make_high_pass_filter, filter_type)

4. Varie o parâmetro de frequência de corte no filtro passa-baixa criado na tarefa 2. Por exemplo, tome valores de D0 iguais a 0,01, 0,05, 0,5. A imagem inicial é igual à anterior. Visualize as imagens dos filtros e as imagens resultantes. Explique os resultados.
- Primeiro foram aplicados os valores de frequência de corte 0.01, 0.05 e 0.5 no filtro passa-baixa ideal. Os resultados são mostrados abaixo:

In [ ]:
for filter_type in filter_types:
    for cutoff in [0.01, 0.05, 0.5]:
        for img_path in os.listdir('./images'):
            process_image(img_path, cutoff, make_low_pass_filter, filter_type)

5.  Efetue o mesmo que se pede no item 4, mas use o filtro passa-alta em vez do filtro passa-baixa.

In [ ]:
for filter_type in filter_types:
    for cutoff in [0.01, 0.05, 0.5]:
        for img_path in os.listdir('./images'):
            process_image(img_path, cutoff, make_high_pass_filter, filter_type)

6. Além dos filtros passa-baixa e passa-alta também existe o filtro passa-banda? Explique seu funcionamento e aplique um filtro passa-banda na imagem.

- O filtro passa-banda é um filtro que permite a passagem de frequências dentro de uma banda de frequência específica. O filtro passa-banda é composto por dois filtros passa-baixa, um com frequência de corte inferior e outro com frequência de corte superior. A imagem resultante é a subtração da imagem filtrada com a frequência de corte inferior da imagem filtrada com a frequência de corte superior. A implementação do filtro passa-banda é mostrada abaixo:


In [ ]:
def make_band_pass_filter(filter_type):
    __name__ = filter_type

    if filter_type == 'butterworth':
        return lambda img, _, __: high_pass_butterworth_filter(img, cutoff_low,
                                                               static_order) * low_pass_butterworth_filter(img,
                                                                                                           cutoff_high,
                                                                                                           static_order)
    elif filter_type == 'gaussian':
        return lambda img, _: high_pass_gaussian_filter(img, cutoff_low) * low_pass_gaussian_filter(img, cutoff_high)
    elif filter_type == 'ideal':
        return lambda img, _: high_pass_ideal_filter(img, cutoff_low) * low_pass_ideal_filter(img, cutoff_high)
    else:
        raise Exception('Invalid filter type')


cutoff_low = 0.05
cutoff_high = 0.1

for filter_type in filter_types:
    for img_path in os.listdir('./images'):
        process_image(img_path, 25, make_band_pass_filter, filter_type)